In [12]:
import numpy as np
import pandas as pd
import os

**FinalProject2025.pdf**

***2.2 Neuronal functional connectivity analysis***

Data: The dataset include 130 simultaneously recorded neuros from the gustatory cortex while the rat learn to associate a novel taste with malaise. The data
is located in the Moodle page (titled "Anan3"). See more instractions there.

Main goal: Estimate the connection strength between the neurons to create
a functional connectivity map.

Tasks:
1. First define the connectivity matrix between neurons. Use some statistic
to test for significant connectivity
2. Compute a measure of the strength of the connection.
3. Use some methods to visualize the connectivity other than a matrix

**instruction from moodle page under "Anan3"**

***Assessing  neuronal  network functional connectivity from spike trains***

**Data**: spiking activity of ensemble of ~130 neurons in the gustatory and somatosensory cortices
recorded continuously (across ~30 hours) as rats learn to associate a novel taste with gastric malaise.

**Your task**: find the connectivity between pairs of neurons in the network to show the changes across
time, and learning. (Hint: cross correlation…)

**Bonus**: use some cool packages for network representation to show the network topology. 

The input files:

cluster_info.tsv – a Tab-separated values file which contains information about the sorted neurons.
Each line contains the information for one optional neuron (sorted cluster). For your analysis only
take neurons (clusters) that their “group” value is “good” (should be around 130).

spike_seconds.npy – a list of spike times (in seconds)

spike_clusters.npy – a list of cluster ids, which correspond to the spikes in spike_seconds.npy

so if you load both files:

cluster_ids = np.load(‘spike_clusters.npy’)

st = np.load('spike_seconds.npy')

if cluster_ids[0] = 5 and st[0] = 11.56, it means that there was a spike for cluster 5 at 11.56 seconds
from the beginning of the recording.

## Step 1: Load and Filter Data
We first load the spike time data and filter it to include only 'good' neurons.

In [17]:
def load_spike_data(spike_times_file, spike_clusters_file, cluster_info_file):
    """Load and filter spike data for 'good' neurons."""
    spike_times = np.load(spike_times_file)  # Load spike times (in seconds)
    spike_clusters = np.load(spike_clusters_file)  # Load neuron IDs

    # Load cluster info and filter for 'good' neurons
    cluster_info = pd.read_csv(cluster_info_file, sep='\t')
    good_neurons = cluster_info[cluster_info['group'] == 'good']['cluster_id'].values

    # Filter spikes for 'good' neurons
    mask = np.isin(spike_clusters, good_neurons)
    filtered_spike_clusters = spike_clusters[mask]
    filtered_spike_times = spike_times[mask]

    return filtered_spike_times, filtered_spike_clusters, good_neurons

In [18]:
# sample output

spike_times, spike_clusters, good_neurons = load_spike_data(
    "spike_seconds.npy", "spike_clusters.npy", "cluster_info.tsv"
)
print("Number of good neurons:", len(good_neurons))
print("\nFirst few spike times:", spike_times[:10])
print("\nFirst few spike clusters:", spike_clusters[:10])

Number of good neurons: 131

First few spike times: [[0.00833343]
 [0.00853344]
 [0.00916678]
 [0.01016679]
 [0.01626686]
 [0.02190027]
 [0.02316695]
 [0.03030037]
 [0.03130038]
 [0.03410042]]

First few spike clusters: [154 923 917 845 112  70 251 954 845 921]


In [19]:
print(len(spike_times), len(spike_clusters))

20530010 20530010


In [20]:
print("spike_times shape:", spike_times.shape)
print("spike_clusters shape:", spike_clusters.shape)

spike_times shape: (20530010, 1)
spike_clusters shape: (20530010,)


In [21]:
spike_times = spike_times.flatten()  # Convert (N,1) → (N,)

In [26]:
def split_into_segments(spike_times, segment_duration):
    """Splits spike times into fixed-size segments."""
    total_duration = np.max(spike_times)
    # num_segments = int(np.ceil(total_duration / segment_duration))
    num_segments = int(np.max(spike_times) // segment_duration)  # Only full segments
    
    segment_ranges = [
        # (i * segment_duration, min((i + 1) * segment_duration, total_duration))
        (i * segment_duration, (i + 1) * segment_duration) 
        for i in range(num_segments)
    ]
    
    return segment_ranges

In [24]:
def bin_spike_trains(spike_times, spike_clusters, neuron_ids, segment_range, bin_size=0.001, binary_mode=False):
    """
    Converts spike timestamps into a fixed-size binned spike train matrix.

    Parameters:
    - binary_mode (bool): If True, store 0/1 (binary); otherwise, count spikes.

    Returns:
    - binned_spike_train (np.array): Fixed-size binned spike matrix (neurons × time bins).
    """
    start_time, end_time = segment_range
    num_bins = int((end_time - start_time) / bin_size)  

    # Select spikes within this segment
    mask = (spike_times >= start_time) & (spike_times < end_time)
    segment_spike_times = spike_times[mask] - start_time  
    segment_spike_clusters = spike_clusters[mask]

    # Initialize spike train matrix (ensuring fixed shape)
    binned_spike_train = np.zeros((len(neuron_ids), num_bins), dtype=np.uint8)

    # Convert spike times to bin indices
    bin_indices = (segment_spike_times / bin_size).astype(int)

    # Map neuron ID to row index for fast lookup
    neuron_idx_map = {neuron: i for i, neuron in enumerate(neuron_ids)}

    # Populate the matrix
    for spike_idx, neuron_id in enumerate(segment_spike_clusters):
        if neuron_id in neuron_idx_map:
            row = neuron_idx_map[neuron_id]
            col = bin_indices[spike_idx], num_bins - 1
            if binary_mode:
                binned_spike_train[row, col] = 1  # Store 0/1 only
            else:
                binned_spike_train[row, col] += 1  # Store spike count

    return binned_spike_train

In [27]:
def process_and_save_segments(spike_times, spike_clusters, neuron_ids, bin_size, segment_duration):
    """
    Processes and saves each fixed-size segment to disk to avoid memory issues.

    Parameters:
    - spike_times (np.array): Array of spike times in seconds.
    - spike_clusters (np.array): Corresponding neuron IDs for each spike.
    - neuron_ids (np.array): Unique neuron identifiers (good neurons).
    - bin_size (float): Time bin size in seconds.
    - segment_duration (int): Duration of each segment in seconds.

    Saves:
    - Each segment's binned spike train as a `.npz` file.
    """
    output_dir = "binned_segments"
    os.makedirs(output_dir, exist_ok=True)

    segment_ranges = split_into_segments(spike_times, segment_duration)
    
    for idx, segment_range in enumerate(segment_ranges):
        print(f"Processing & Saving Segment {idx + 1}/{len(segment_ranges)}: {segment_range[0]}s to {segment_range[1]}s")

        # Bin spike trains for the segment
        binned_data = bin_spike_trains(spike_times, spike_clusters, neuron_ids, segment_range, bin_size)

        # Save to compressed .npz file
        np.savez_compressed(
            os.path.join(output_dir, f"binned_segment_{idx}.npz"),
            spike_train=binned_data,
            neuron_ids=neuron_ids
        )

        print(f"✅ Segment {idx + 1} saved: {output_dir}/binned_segment_{idx}.npz")

# Run processing with memory-efficient saving
bin_size = 0.001  # 1 ms = 1000 Hz binning
segment_duration = 7200  # 2 hours = 7200 seconds

process_and_save_segments(spike_times, spike_clusters, good_neurons, bin_size, segment_duration)

print("✅ All segments processed and saved to disk.")

Processing & Saving Segment 1/15: 0s to 7200s
✅ Segment 1 saved: binned_segments/binned_segment_0.npz
Processing & Saving Segment 2/15: 7200s to 14400s
✅ Segment 2 saved: binned_segments/binned_segment_1.npz
Processing & Saving Segment 3/15: 14400s to 21600s
✅ Segment 3 saved: binned_segments/binned_segment_2.npz
Processing & Saving Segment 4/15: 21600s to 28800s
✅ Segment 4 saved: binned_segments/binned_segment_3.npz
Processing & Saving Segment 5/15: 28800s to 36000s
✅ Segment 5 saved: binned_segments/binned_segment_4.npz
Processing & Saving Segment 6/15: 36000s to 43200s
✅ Segment 6 saved: binned_segments/binned_segment_5.npz
Processing & Saving Segment 7/15: 43200s to 50400s
✅ Segment 7 saved: binned_segments/binned_segment_6.npz
Processing & Saving Segment 8/15: 50400s to 57600s
✅ Segment 8 saved: binned_segments/binned_segment_7.npz
Processing & Saving Segment 9/15: 57600s to 64800s
✅ Segment 9 saved: binned_segments/binned_segment_8.npz
Processing & Saving Segment 10/15: 64800s t

In [29]:
from scipy.signal import correlate

def compute_cross_correlation(binned_spike_train, max_lag=40):
    """
    Computes the cross-correlation function for all neuron pairs.

    Parameters:
    - binned_spike_train (np.array): Shape (num_neurons, num_bins), 0/1 values.
    - max_lag (int): Max time lag in milliseconds.

    Returns:
    - cross_corr_matrices (dict): Dictionary where keys are neuron pairs (i, j),
      values are the cross-correlation functions.
    """
    num_neurons = binned_spike_train.shape[0]
    cross_corr_matrices = {}

    for i in range(num_neurons):
        for j in range(num_neurons):
            if i != j:  # Skip self-correlation
                corr = correlate(binned_spike_train[i], binned_spike_train[j], mode='full')
                lag_center = len(corr) // 2
                cross_corr_matrices[(i, j)] = corr[lag_center - max_lag : lag_center + max_lag]  # ±max_lag region

    return cross_corr_matrices


In [30]:
# Load a segment
segment_idx = 0
file_path = f"binned_segments/binned_segment_{segment_idx}.npz"
data = np.load(file_path)
binned_spike_train = data["spike_train"]

# Compute cross-correlation
cross_corr_matrices = compute_cross_correlation(binned_spike_train)